In [1]:
from aco import ACO
import numpy as np
import torch
import logging

import numpy as np

N_ANTS = 10
N_ITERATIONS = [1, 10, 20, 30, 40, 50]

def heuristics_reevo(item_attr1: np.ndarray, item_attr2: np.ndarray) -> np.ndarray:
    n, m = item_attr2.shape
    
    # Normalize item_attr1 and item_attr2
    item_attr1_norm = (item_attr1 - np.min(item_attr1)) / (np.max(item_attr1) - np.min(item_attr1))
    item_attr2_norm = (item_attr2 - np.min(item_attr2)) / (np.max(item_attr2) - np.min(item_attr2))
    
    # Calculate the average value of normalized attribute 1
    avg_attr1 = np.mean(item_attr1_norm)
    
    # Calculate the maximum value of normalized attribute 2 for each item
    max_attr2 = np.max(item_attr2_norm, axis=1)
    
    # Calculate the sum of normalized attribute 2 for each item
    sum_attr2 = np.sum(item_attr2_norm, axis=1)
    
    # Calculate the standard deviation of normalized attribute 2 for each item
    std_attr2 = np.std(item_attr2_norm, axis=1)
    
    # Calculate the heuristics based on a combination of normalized attributes 1 and 2,
    # while considering the average, sum, and standard deviation of normalized attribute 2
    heuristics = (item_attr1_norm / max_attr2) * (item_attr1_norm / avg_attr1) * (item_attr1_norm / sum_attr2) * (1 / std_attr2)
    
    # Normalize the heuristics to a range of [0, 1]
    heuristics = (heuristics - np.min(heuristics)) / (np.max(heuristics) - np.min(heuristics))
    
    return heuristics


def solve(prize: np.ndarray, weight: np.ndarray):
    n, m = weight.shape
    heu = heuristics_reevo(prize.copy(), weight.copy()) + 1e-9
    assert heu.shape == (n,)
    heu[heu < 1e-9] = 1e-9
    aco = ACO(torch.from_numpy(prize), torch.from_numpy(weight), torch.from_numpy(heu), N_ANTS)
    
    results = []
    for i in range(len(N_ITERATIONS)):
        if i == 0:
            obj, _ = aco.run(N_ITERATIONS[i])
        else:
            obj, _ = aco.run(N_ITERATIONS[i] - N_ITERATIONS[i-1])
        results.append(obj.item())
    return results


In [2]:
for problem_size in [100]:
    dataset_path = f"dataset/val{problem_size}_dataset.npz"
    dataset = np.load(dataset_path)
    prizes, weights = dataset['prizes'], dataset['weights']
    n_instances = prizes.shape[0]
    logging.info(f"[*] Evaluating {dataset_path}")

    objs = []
    for i, (prize, weight) in enumerate(zip(prizes, weights)):
        obj = solve(prize, weight)
        objs.append(obj)
    
    # Average objective value for all instances
    mean_obj = np.mean(objs, axis=0)
    for i, obj in enumerate(mean_obj):
        print(f"[*] Average for {problem_size}, {N_ITERATIONS[i]} iterations: {obj}")
    print()

[*] Average for 100, 1 iterations: 16.183777390580715
[*] Average for 100, 10 iterations: 16.95788758133592
[*] Average for 100, 20 iterations: 17.068556032421405
[*] Average for 100, 30 iterations: 17.113623508631083
[*] Average for 100, 40 iterations: 17.113623508631083
[*] Average for 100, 50 iterations: 17.137104057227354

